In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.models import load_model
import keras
import pandas as pd
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import os
import random
import pickle
import seaborn as sns
import timeit
from sklearn.metrics import confusion_matrix
import itertools 

In [ ]:
#Initialize List to Append each data file

DATA_PATH = '/datasets/home/21/321/ee228sp20ta1/G36/'
#'Different Data Files' for training,  validation adn test
filenames=['processed_data.pickle', 'train.pickle']
# i=0
data=[]
def show_data(data, num_exp):
    '''
    Display some random instances
    of input data.
    
    Input:-
    -----
    data :  type(4-D array)
        Input data in the form
        of images
        
    '''
    assert 0<=num_exp<=5

    x=[np.array([i]) for i in np.arange(data.shape[0])]
    rnd_ind = random.choices(x, k = num_exp)
    images = (data[rnd_ind,:,:,:]).squeeze(1)
    h_stack_image =np.hstack(images) # Getting Incorrect Images and Stacking
                                 # them to print in one go
    plt.figure(figsize = (30,6))
    plt.imshow(h_stack_image, cmap='gray')    # Converting into unsigned integers for plt.imshow              
    plt.xticks([])
    plt.yticks([])  
    
    
def show_hist(labels, status, color):
    plt.hist(labels, bins = 43,color= color,rwidth= 0.9 )
    plt.xlabel('Classes', fontsize = '15')
    plt.ylabel('Frequency', fontsize = '15')
    plt.title('Classes Distribution '+ status+ ' Preprocessing', fontsize = '15')
    
    
for i in range(2):
        file= os.path.join(DATA_PATH, filenames[i])
        with open(file, 'rb') as f:
             data.append(pickle.load(f, encoding='latin1'))  

#Plotting classes distribution before  and after preprocessing       
pro=data[0]['y_train'][:,None]
pre=data[1]['labels'][:,None]
plt.figure(figsize = (20,7))
plt.subplot(1,2,1)
show_hist(pre, 'Before', 'r')
plt.subplot(1,2,2)
show_hist(pro, 'After', 'g')

In [ ]:
# Make dictionary of the stored csv files from each of the model fitting history
def make_dict_model(net):
    '''
    Read csv file defined by the model name
    Input
    model name (string)
    
    Returns
    dict{}
    Keys:
    epoch: Maximum number of epochs on which it was trained
    indices: indices for plotting accuracy from 0 to epoch with step size = 10
    vals: values of validation accuracy at those indices
    train: maximum accuracy achieved on the training set
    val:   maximum accuracy achieved on the validation set

    '''
    assert type(net)==str
    indx=list(range(1,102,9))
    vals=list(range(0,101,9))
    dictionary={}
    filename= './outputs/model_'+net+'.csv'
    net=pd.read_csv(filename)
   
    if 'val_acc' in net.columns:
        
        dictionary={'epoch': len(net['val_acc']), 'indices':indx, 
                   'val_acc': net['val_acc'].iloc[vals], 'val':net['val_acc'][99], 'train':net['acc'][99]}
    elif 'val_accuracy' in net.columns:
         dictionary={'epoch': len(net['val_accuracy']), 'indices':indx, 
                   'val_acc': net['val_accuracy'].iloc[vals], 'val':net['val_accuracy'][99], 'train':net['accuracy'][99]}
    else:
        dictionary= {'epoch': len(net['val_output_accuracy']), 'indices':indx, 
        'val_acc': net['val_output_accuracy'].iloc[vals], 'val':net['val_output_accuracy'][99], 'train':net['output_accuracy'][99]}


    return dictionary

densenet_dict=make_dict_model('DensNet') 
resnet_dict=make_dict_model('ResNet') 
inception_dict=make_dict_model('GoogleNet') 
ours_dict=make_dict_model('new_impl')
VGG19_dict=make_dict_model('VGG19')




In [ ]:
#Make Scatter plot for validation accuracy
plt.figure(figsize = (15,7))

plt.scatter(densenet_dict['indices'], densenet_dict['val_acc'], marker= 'D',color= 'r', s= 100, label = 'DenseNet')
plt.scatter(resnet_dict['indices'], resnet_dict['val_acc'], marker= '8',color= 'b', s= 90,label = 'ResNet')
plt.scatter(ours_dict['indices'], ours_dict['val_acc'], marker= '*',color= 'y', s= 120,label = 'OurModel')
plt.scatter(inception_dict['indices'], inception_dict['val_acc'], marker= '>', s= 80, color= 'darkslategray', label = 'Inceptionv1')
plt.scatter(rnn_dict['indices'], rnn_dict['val_acc'], marker= 'd',color= 'darkred', s= 70,label = 'RNN')
plt.scatter(VGG19_dict['indices'], VGG19_dict['val_acc'], marker= '<',color= 'maroon', s= 70,label = 'VGG-19')


plt.xlabel('Epochs', fontsize =15)
plt.ylabel('Test Accuracy', fontsize =15)
plt.xlim([0,101])
plt.ylim([0.85,1.00])
plt.legend(fontsize=12)




In [ ]:
#Make Line Plot for validation Accuracy
'''
Color Scheme
        color=['forestgreen'],  label= 'Our Model
        color=['magenta'],  label = 'VGG19'
        color=['firebrick'], label= 'ResNet'
        color=['goldenrod'], label = 'Densenet'
        color=['darkorange'],label = 'Inception-V1'
        color=['blue'],  label ='RNN'

'''
plt.figure(figsize = (9,7))

plt.plot(densenet_dict['indices'], densenet_dict['val_acc'], marker= 'D',color= 'goldenrod', label = 'DenseNet')
plt.plot(resnet_dict['indices'], resnet_dict['val_acc'], marker= '8',color= 'firebrick', label = 'ResNet')
plt.plot(ours_dict['indices'], ours_dict['val_acc'], marker= '*',color= 'forestgreen', label = 'OurModel')
plt.plot(inception_dict['indices'], inception_dict['val_acc'], marker= '>',  color= 'darkorange', label = 'Inceptionv1')
plt.plot(rnn_dict['indices'], rnn_dict['val_acc'], marker= 'd',color= 'blue', label = 'RNN')
plt.plot(VGG19_dict['indices'], VGG19_dict['val_acc'], marker= '<',color= 'm', label = 'VGG-19')


plt.xlabel('Epochs', fontsize =15)
plt.ylabel('Validation Accuracy', fontsize =15)
plt.xlim([0,101])
plt.ylim([0,1.1])
plt.legend(fontsize=12)


In [ ]:
#Plot Confusion Matrix plot from confusion matrix numpy array
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure(figsize=(20,10))
#Make Histograms for showing train, test, validation accuracies of each model
'''
Color Scheme
        color=['forestgreen'],  label= 'Our Model
        color=['teal'],  label = 'VGG19'
        color=['firebrick'], label= 'ResNet'
        color=['goldenrod'], label = 'Densenet'
        color=['darkorange'],label = 'Inception-V1'
        color=['blue'],  label ='RNN'

'''


kwargs = {'alpha':0.8, 'linestyle':':', 'linewidth':3, 'edgecolor':'k'}
plt.bar(x=np.arange(0,6,1), height= 100*np.array([ours_dict['train'], VGG19_dict['train'], resnet_dict['train'],densenet_dict['train'] ,inception_dict['train'] ,rnn_dict['train']]), color=['forestgreen', 'teal','firebrick','goldenrod','darkorange', 'blue'], width = 0.9, **kwargs)
plt.bar(x= np.arange(8,14,1), height= 100*np.array([ours_dict['val'], VGG19_dict['val'], resnet_dict['val'],densenet_dict['val'],  inception_dict['val'],rnn_dict['val']]),color=['forestgreen', 'teal','firebrick','goldenrod','darkorange', 'blue'], width = 0.9, **kwargs)

plt.bar(x= 16, height=  0.9771*100, width = 0.9, color=['forestgreen'], **kwargs, label= 'Our Model')
plt.bar(x=17, height=  0.9812*100, width = 0.9, color=['teal'], **kwargs, label = 'VGG19')
plt.bar(x= 18, height= 0.9480*100, width = 0.9, color=['firebrick'], **kwargs, label= 'ResNet')
plt.bar(x= 19, height= 0.9559*100, width = 0.9, color=['goldenrod'], **kwargs, label = 'Densenet')
plt.bar(x=20, height=  0.9516*100, width = 0.9, color=['darkorange'], **kwargs, label = 'Inception-V1')
plt.bar(x= 21, height=  0.89*100, width = 0.9, color=['blue'], **kwargs, label ='RNN')


plt.xticks(np.arange(2.5,22,8), ['Training Accuracy', 'Validation Accuracy', 'Test Accuracy' ], fontsize = 20)

plt.ylim([80,105])
plt.ylabel('Accuracy(%)', fontsize = 20, color = 'k')
plt.legend( fontsize = 15)


In [ ]:
# Calculate evaluation time for each model for the whole test set
model=tf.keras.models.load_model('model_ours_data')
%timeit model.predict(data[0]['x_test'])

In [ ]:
vgg= load_model('./trained_models/model_VGG19_data')
%timeit vgg.predict(data[0]['x_test'])

In [ ]:
rnn= load_model('./trained_models/model_RNN_data')
%timeit rnn.predict(data[0]['x_test'])

In [ ]:
inception= load_model('./trained_models/model_GoogleNet_data')
%timeit inception.predict(data[0]['x_test'])

In [ ]:
inception= tf.keras.models.load_model('./trained_models/model_ResNet_data')
%timeit inception.predict(data[0]['x_test'])

In [ ]:
inception= tf.keras.models.load_model('./trained_models/model_DensNet_data')
%timeit inception.predict(data[0]['x_test'])

In [ ]:
#Plot evaluation time for each of the model for the whole set in seconds
'''
Color Scheme
        color=['forestgreen'],  label= 'Our Model
        color=['teal'],  label = 'VGG19'
        color=['firebrick'], label= 'ResNet'
        color=['goldenrod'], label = 'Densenet'
        color=['darkorange'],label = 'Inception-V1'
        color=['blue'],  label ='RNN'

'''
plt.figure(figsize=(10,10))

kwargs = {'alpha':0.8, 'linestyle':':', 'linewidth':3, 'edgecolor':'k'}
plt.barh(width=[1.44,4.32, 4.37, 5.11,7.23, 15.4],  y=np.arange(6), 
         color=['forestgreen','teal','darkorange','firebrick', 'goldenrod', 'blue'], **kwargs)
plt.yticks(np.arange(6), ['OurModel', 'VGG19', 'Inception-V1','ResNet','DenseNet', 'RNN'], fontsize = 15)
plt.xlabel('Prediction Time (s)', fontsize = 15)
plt.ylim([-0.5,5.5])


In [ ]:
#Plot Confusion Matrix for our model
y=pd.read_csv('./outputs/our_model_prediction.csv')

y_predict=y.iloc[0]
y_test=y.iloc[1]
  
cmt=confusion_matrix(y_predict, y_test)


In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')


    plt.imshow(cm, interpolation='nearest', cmap='YlGnBu')
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="red" if cm[i, j] > thresh else "black", fontsize = 11)

    plt.tight_layout()
    plt.ylabel('True label', fontsize = 15)
    plt.xlabel('Predicted label', fontsize = 15)
    
plt.figure(figsize=(12, 12))
plot_confusion_matrix(cmt, classes=np.arange(43))
plt.title('Confusion Plot for our Model', fontsize='15')
plt.savefig('confusion_matrix.png')